In [ ]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [ ]:
dataset,info=tfds.load('StanfordDogs', as_supervised=True,with_info=True)

In [ ]:
batch_size=32
data_size=info.splits['train'].num_examples
class_names=info.features['label'].names
n_classes=info.features['label'].num_classes
test_size=info.splits['test'].num_examples

In [ ]:
test_set,valid_set,train_set=tfds.load(
    'StanfordDogs',
    as_supervised=True,
    split=['test','train[:20%]','train[20%:]']
)

In [ ]:
tfds.visualization.show_examples(train_set,info)

# Preprocessing data

In [ ]:
def preprocess(image,label):
    resized_image=tf.image.resize(image,[224,224])
    final_image=keras.applications.resnet50.preprocess_input(resized_image)
    return final_image,label

In [ ]:
train_set=train_set.shuffle(1000)
train_set=train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set=valid_set.map(preprocess).batch(batch_size).prefetch(1)
test_set=test_set.map(preprocess).batch(batch_size).prefetch(1)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_set.take(1):
      for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

# Building model using transfer learning from ResNet50 model

In [ ]:
base_model=keras.applications.resnet50.ResNet50(weights='imagenet',include_top=False)
avg=keras.layers.GlobalAveragePooling2D()(base_model.output)
output=keras.layers.Dense(n_classes,activation="softmax")(avg)
model=keras.Model(inputs=base_model.input,outputs=output)

In [ ]:
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
optimizer=keras.optimizers.SGD(learning_rate=0.2,momentum=0.9,decay=0.01)
model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
early_stopping=keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)

In [ ]:
history=model.fit(train_set,epochs=10,validation_data=valid_set)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
for layer in base_model.layers:
  layer.trainable=True

In [ ]:
optimizer=keras.optimizers.SGD(learning_rate=0.01,momentum=0.9,decay=0.001,nesterov=True)
history=model.fit(train_set,epochs=200,validation_data=valid_set,callbacks=[early_stopping])

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
model.evaluate(test_set)

In [ ]:
y_pred=model.predict(test_set)

In [ ]:
k=test_set.unbatch()

In [ ]:
false_pred={class_name:0 for class_name in class_names}
true_pred={class_name:0 for class_name in class_names}
top_1=0
top_5=0
for pred,(images,label) in zip(y_pred,k):
    if np.argmax(pred)!=label.numpy():
        false_pred[class_names[label.numpy()]]=false_pred[class_names[label.numpy()]]+1
        if label.numpy() in np.argpartition(pred, -4)[-4:]:
            top_5+=1
    else:
        top_5+=1
        top_1+=1
        true_pred[class_names[label.numpy()]]=true_pred[class_names[label.numpy()]]+1


In [ ]:
benchmark=pd.DataFrame(data={'true_pred':true_pred.values(),'false_pred':false_pred.values()},index=class_names)

In [ ]:
benchmark['% bad_prediction']=np.int64(benchmark.false_pred*100/(benchmark.true_pred+benchmark.false_pred))
benchmark['% good_prediction']=np.int64(benchmark.true_pred*100/(benchmark.true_pred+benchmark.false_pred))

In [ ]:
benchmark

In [ ]:
benchmark.nlargest(5,'% bad_prediction')

In [ ]:
benchmark.nlargest(5,'% good_prediction')

In [ ]:
print('TOP 1 ACCURACY:',top_1/test_size)
print('TOP 5 ACCURACY:',top_5/test_size)

In [ ]:
model.save('dog_classif_model.h5')

In [ ]:
model_2=keras.models.load_model('dog_classif_model.h5')

In [ ]:
np.savetxt("class_names.csv", np.array(class_names,dtype='str'),delimiter=",",fmt='%s')

In [ ]:
def preprocess_new(image):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.resnet50.preprocess_input(resized_image)
    return final_image

In [ ]:
from PIL import Image
img=Image.open('../input/solos-data/chihuahua-2900362__480.jpg')
img_to_tensor = tf.convert_to_tensor(img)
a=preprocess_new(img_to_tensor)
prediction=model(a.numpy().reshape(1,224,224,3),training=False)